In [1]:
# For the simulation of a single server queue it is important to remember that the main events will be arrival and departure. We will be monitoring the flow of the people in the queue and the server.

# Importing the necessary libraries.
import math
import random

# Defining constants.
Q_LIMIT = 100
BUSY = 1
IDLE = 0

# We declare the variable and initialize these varaiables to zero.
next_event_type, num_custs_delayed, num_delays_required, num_events, num_in_q, server_status = 0,0,0,0,0,0

area_num_in_q, area_server_status, mean_interarrival, mean_service, sim_time, time_last_event, total_of_delays = 0.0,0.0,0.0,0.0,0.0,0.0,0.0
time_arrival = [0] * (Q_LIMIT+1)


# An array to implement event list
time_next_event = [0] * 3

# An initialization function for initializing variables.
def initialization():
    
    # Some variables such as sim_time, servers_status, num_in_q, time_last_event
    # num_custs_delayed, total_of delays, area_num_q and area_server_status are already initialized above
    
    
    """Time for the first arrival, time_next_event[1] is determined by adding an exponential random variate with 
       mean mean_interarrival, namely, expon(mean_interarrival) to the simulation clock.
       Though sim_time at this point is zero, we are using it to show the general form of a statement
       to determine the time of a future event.
    """
    
    time_next_event[1] = sim_time + expon(mean_interarrival)
    
    """Since no customer are present at time sim_time = 0, the time of the next departure, time_next_event[2], 
       is set to 10e + 30, guaranteeing that the first event will be an arrival.
       This is because the value of the next event, departure, has been set too high such that we're quarenteed
       that person will have come before this time is exhausted. 
    """
    time_next_event[2] = 1.0 * 10**30

# Declaring the timing function.

""""A timing function is used to compare time_next_event[1], time_next_event[2] ..., time_next_event[num_events].
    This means that we are comparing the timing for these consequent events for instance the time for the arrival, 
    followed by a departure followed by an arrival and so of depending on the occurrence of the events.
    Incase of ties, the lowest-numbered event type is choosen. Then the simualation is advanced to the time of occurrence
    of the chosen event type,  and stored in this varible, min_time_next_event.
"""
def timing():
    
    # Some function global variables. 
    global sim_time, next_event_type, num_events

    main_time_next_event = 1.0 * 10**29
    next_event_type = 0
    
    # This for loop determines the event type of the next event to occur.
    for i in range(num_events):
        # If the time_next_event at position [i+1] is less that the main_time_next event then the main_time_next_event 
        # is assigned that value.
        if (time_next_event[i+1] < main_time_next_event):
            main_time_next_event = time_next_event[i+1]
            next_event_type = i+1
    
    # This if statement check to see if the the event list is empty. If it is the we stop the simulation clock. 
    if (next_event_type == 0):
        output_file.write(
            "\nEvent list is empty at time {0}".format(sim_time))
        exit(1)
        
    
    # If the event list is not empty, then we advance the simulation clock.
    # The simulation clock is advanced to the time of occurrence of the chosen event type.
    sim_time = main_time_next_event

# Arrival event fnction.
def arrive():
    
    # Some global variables for the function.
    global server_status, num_in_q, total_of_delays, num_custs_delayed, sim_time, mean_interarrival, time_next_event, Q_LIMIT, time_arrival, mean_service
    
    # Initializing the a float variable, delay to zero.
    delay = 0.0
    
    # Scheduling the next arrival.
    time_next_event[1] = sim_time + expon(mean_interarrival)
    
    # The firsts if function checks if the server is busy.
    if (server_status == BUSY):
        # If the server is busy, we increment the number of customers in the queue.
        num_in_q += 1
        
        #This if stamement checks for an overflow, that is, number of customer exceeds the queue limit we defined.
        if(num_in_q > Q_LIMIT):
            # If the queue has overflown, we stop the simulatiom.
            output_file.write("\nOverflow of the array time_interval at ")
            output_file.write("time {0}".format(sim_time))
            exit(2)
            
        # There is still room in the queue, so we store the time of arrival of the arriving customer at the (new) end of 
        # time_arrival.
        time_arrival[num_in_q] = sim_time
        
    # The server is IDLE so the arriving customer has a delay of zero. That is, once they arrive they directly go for 
    # service.
    else:
        delay = 0.0
        #total_of_delays += delay
        
        # Increment the number of customers delayed and make server busy
        num_custs_delayed += 1
        
        # The server now acquires a new state as there is someone in service.
        server_status = BUSY

        # Schedule a departure, that is, service completion.
        time_next_event[2] = sim_time + expon(mean_service)

        
# The departure event function.
def depart():

    # Defining some global variables.
    global total_of_delays, num_custs_delayed, num_in_q, server_status, time_next_event, sim_time, time_arrival, mean_service
    delay = 0.0
    
    # Checking to see if the queue is empty.
    if (num_in_q == 0):
        
        # If the queue is empty we make the server IDLE and eliminate the departure(service completion) event from 
        # consideration. 
        server_status = IDLE
        
        # We then set the departure time (next event so high such that an arrival is quarenteed first)
        # Logically speaking we cannot have a depature if the queue is empty if we assume that the server is also not busy.
        time_next_event[2] = 1.0 * 10**30
        
    # Otherwise if the queue is not empty.    
    else:
        
        # We decrement the number of customers in the queue.
        num_in_q -= 1
        
        # Compute the delay of the customer who is beginning service and update the total delay accumulator.
        # This is done by taking the time a person waits before joining the queue and the time they came.
        delay = sim_time - time_arrival[1]
        total_of_delays += delay
        
        # Increment the number of customers delayed and schedule departure.
        num_custs_delayed += 1
        time_next_event[2] = sim_time + expon(mean_service) # Remember we had this defined formula for the departure earlier.
        
        # Move each customer in queue(if any) up one place, this is because a customer have just left therefore the people 
        # in queue have reduced.
        for i in range(num_in_q):
            """
            Set time arrival for next customer to the next customer + 1. 
            Don't start at time_arrival[i]
            """
            time_arrival[i+1] = time_arrival[i + 2]

# A report generator function.
def report():

    # Defining some global functions.
    global total_of_delays, num_custs_delayed, sim_time, area_num_in_q, area_server_status
    
    # Compute and write estimates of desired measures of perfomance.
    print(f"\nAverage delay in queue {round((total_of_delays/num_custs_delayed),4)} minutes\n\n", file=output_file)
    print(f"Average number in queue  {round((area_num_in_q/sim_time),4)}\n\n", file=output_file)
    print(f"Server utilization  {round((area_server_status/sim_time),4)}\n\n", file=output_file)
    print(f"Time simulation ended  {round(sim_time,4)} \n\n", file=output_file)


"""
   Update area accumulator for time-average statistics.
   This function is invoked just before processing each event (of any type) and updates the areas under 
   the two functions needed for the continous-time statistics.
   This is not an event routine.
"""
def update_time_avg_stats():

    # Defining some global variables.
    global time_last_event, sim_time, area_num_in_q, area_server_status, num_in_q, server_status

    time_since_last_event = 0.0

    # Compute time since last event and update last-event-time marker.
    time_since_last_event = sim_time - time_last_event
    time_last_event = sim_time
    
    # Updating the area under number-in-queue function.
    area_num_in_q += num_in_q * time_since_last_event
    
    # Updating area under server_status indicator function.
    area_server_status += server_status * time_since_last_event


# An Exponential variate generation function.
def expon(mean):
    
    # Return an exponential random variate with mean "mean".
    # We use the random.random() to generate the random numbers.

    rand = random.random()
    return -(float(mean)) * math.log(rand)


# The Main function definition.
if __name__ == '__main__':
    
    # Opening the input and output files.
    input_file = open("infile.txt", "r")
    output_file = open("outfile.txt", "r+") 
    
    # Specify the number of events for the timing function.
    num_events = 2

    # Reading input parameters.
    input_parameters = input_file.readline().split()

    # The three parameters in the infile.txt file i.e, 1.0,0.5,1000.
    mean_interarrival = input_parameters[0]
    mean_service = input_parameters[1]
    num_delays_required = input_parameters[2]

    # Write report heading and input parameters.
    output_file.write("Single-server queueing system\n\n")
    output_file.write("Mean interarrival time {0} \n\n".format(mean_interarrival))
    output_file.write("Mean service time {0} \n\n".format(mean_service))
    output_file.write("No of Cutomers {0} \n".format(num_delays_required))

    # Initialize the simulation.
    initialization()

    # Run the simulation while more delays are still needed. 
    # We continously loop until the 1000 stoppage rule given is met.
    while (num_custs_delayed < int(num_delays_required)):

        # Determine the next event
        timing()
        
        # Update time_average statistical accumulators.
        update_time_avg_stats()
        
        # Invoke the appropriate event functions.
        if (next_event_type == 1):
            arrive()
        elif (next_event_type == 2):
            depart()

    # Invoke the report generator and end the simulation.
    report()

    # Return file pointer back to beginning of file
    output_file.seek(0)

    # Output information from file to screen 
    print(output_file.read)

    input_file.close()
    output_file.close()
    

<built-in method read of _io.TextIOWrapper object at 0x000001FDE0583208>
